In [58]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import neural_network, preprocessing
from sklearn.preprocessing import StandardScaler
np.random.seed(42)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
pd.set_option('max_columns', None)

In [59]:
def design_matrix(df):

    # one-hot encodings

    # label encodings
    le_original_payment_type = preprocessing.LabelEncoder()
    le_original_payment_type.fit(df.original_payment_type)
    df.original_payment_type = le_original_payment_type.transform(df.original_payment_type.values)

    le_original_payment_method = preprocessing.LabelEncoder()
    le_original_payment_method.fit(df.original_payment_method)
    df.original_payment_method = le_original_payment_method.transform(df.original_payment_method.values)

    le_charge_option = preprocessing.LabelEncoder()
    le_charge_option.fit(df.charge_option)
    df.charge_option = le_charge_option.transform(df.charge_option.values)

    le_hotel_brand_code = preprocessing.LabelEncoder()
    le_hotel_brand_code.fit(df.hotel_brand_code)
    df.hotel_brand_code = le_hotel_brand_code.transform(df.hotel_brand_code.values)

    le_hotel_chain_code = preprocessing.LabelEncoder()
    le_hotel_chain_code.fit(df.hotel_chain_code)
    df.hotel_chain_code = le_hotel_chain_code.transform(df.hotel_chain_code.values)

    le_hotel_area_code = preprocessing.LabelEncoder()
    le_hotel_area_code.fit(df.hotel_area_code)
    df.hotel_area_code = le_hotel_area_code.transform(df.hotel_area_code.values)

    le_accommadation_type_name = preprocessing.LabelEncoder()
    le_accommadation_type_name.fit(df.accommadation_type_name)
    df.accommadation_type_name = le_accommadation_type_name.transform(df.accommadation_type_name.values)

    le_original_payment_currency = preprocessing.LabelEncoder()
    le_original_payment_currency.fit(df.original_payment_currency)
    df.original_payment_currency = le_original_payment_currency.transform(df.original_payment_currency.values)

    le_customer_nationality = preprocessing.LabelEncoder()
    le_customer_nationality.fit(df.customer_nationality)
    df.customer_nationality = le_customer_nationality.transform(df.customer_nationality.values)

    le_cancellation_policy_code = preprocessing.LabelEncoder()
    le_cancellation_policy_code.fit(df.cancellation_policy_code)
    df.cancellation_policy_code = le_cancellation_policy_code.transform(df.cancellation_policy_code.values)

    le_origin_country_code = preprocessing.LabelEncoder()
    le_origin_country_code.fit(df.origin_country_code)
    df.origin_country_code = le_origin_country_code.transform(df.origin_country_code.values)

    le_hotel_country_code = preprocessing.LabelEncoder()
    le_hotel_country_code.fit(df.hotel_country_code)
    df.hotel_country_code = le_hotel_country_code.transform(df.hotel_country_code.values)

    # booleans
    df['is_first_booking'] = df['is_first_booking'].astype(int)
    df['is_user_logged_in'] = df['is_user_logged_in'].astype(int)
    df[df.request_nonesmoke.isnull()] = 0
    df[df.request_earlycheckin.isnull()] = 0
    df[df.request_highfloor.isnull()] = 0
    df[df.request_largebed.isnull()] = 0
    df[df.request_twinbeds.isnull()] = 0
    df[df.request_airport.isnull()] = 0
    df[df.request_latecheckin.isnull()] = 0

    # columns dropped
    df.drop(['h_booking_id',
             'hotel_id',
             'hotel_live_date',
             'h_customer_id',
             'language',
             'hotel_city_code',
             'guest_nationality_country_name'
             ], axis=1, inplace=True)

    # dates
    def handle_date(col_name):
        df[col_name] = df[col_name].apply(lambda x: np.nan if x == '0000-00-00' or x== np.nan else x)
        df[col_name] = pd.to_datetime(df[col_name], errors='coerce')
        if col_name != 'cancellation_datetime':
            df[col_name + '_month'] = pd.to_numeric(df[col_name].dt.month)
            df[col_name + '_date_day'] = pd.to_numeric(df[col_name].dt.day)
            df[col_name + '_weekday'] = pd.to_numeric(df[col_name].dt.weekday)
            df[col_name + '_year'] = pd.to_numeric(df[col_name].dt.year)
            df[col_name + '_quarter'] = pd.to_numeric(df[col_name].dt.quarter)
        # print(col_name)
        # print(df[col_name])
        # # if(col_name!='cancellation_datetime'):
        # #     df[col_name + '_epoch'] = df[col_name].apply(lambda x: pd.Timestamp.timestamp(x) - 1500000000 )

    handle_date('booking_datetime')
    handle_date('checkout_date')
    handle_date('checkin_date')
    handle_date('cancellation_datetime')

    # get date differences
    df['sub_checkout_checkin'] = (df['checkout_date'].dt.round('1d') - df['checkin_date'].dt.round('1d')) / np.timedelta64(1, 'D')
    df['sub_checkin_booking'] = (df['checkin_date'].dt.round('1d') - df['booking_datetime'].dt.round('1d')) / np.timedelta64(1, 'D')
    df[df['sub_checkin_booking'] == -1] = 0

    # add feature to check if booking includes weekend
    df['bus_days'] = np.busday_count(df['checkin_date'].values.astype('datetime64[D]'), df['checkout_date'].values.astype('datetime64[D]'))
    df['weekends'] = df['sub_checkout_checkin'] - df['bus_days']

    # clean up
    # df.drop(['booking_datetime', 'checkout_date', 'checkin_date'], axis=1, inplace=True)

    # df['sub_cancel_checkin'] = (df['cancellation_date'].dt.round('1d') - df['checkin_date'].dt.round('1d')) / np.timedelta64(1, 'D')

    return df


In [60]:
def train_classifier_model(X, Y):
    # normalize data
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    #fit model
    model = sklearn.neural_network.MLPClassifier(random_state=1, max_iter=500, hidden_layer_sizes=(50,10))
    model.fit(X, Y)
    
    # model = neural_network.MLPRegressor

    return model

def train_regressor_model(X, Y):
    # normalize data
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    #fit model
    model = sklearn.neural_network.MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=(50,10))
    model.fit(X, Y)
    
    # model = neural_network.MLPRegressor

    return model

In [62]:
# Get features
train_set = pd.read_csv("../datasets/agoda_cancellation_train.csv")
labels = train_set['cancellation_datetime'].isnull().astype(int).replace({0:1, 1:0})
# train_set.drop(['cancellation_datetime'], axis=1, inplace=True)
features = design_matrix(train_set)
features['cancellation_datetime'] = pd.to_datetime(features['cancellation_datetime'], errors='coerce')
features['checkout_date'] = pd.to_datetime(features['checkout_date'], errors='coerce')
features['checkin_date'] = pd.to_datetime(features['checkin_date'], errors='coerce')
features['booking_datetime'] = pd.to_datetime(features['booking_datetime'], errors='coerce')


In [88]:
# cancellation date - booking date
def get_cancellation_date_diff(df): 
    date_sub = (df[df['cancellation_datetime'].notna()]['cancellation_datetime'] - df[df['cancellation_datetime'].notna()]['booking_datetime'])
    # print(date_sub.dt.days)
    no_cancel = df['cancellation_datetime'].isnull().astype(int)
    no_cancel = no_cancel.replace({0:-1, 1:0})
    # print(no_cancel)
    # merge date_sub with no_cancel where date_sub is defined
    return pd.concat([no_cancel, date_sub.dt.days], axis=1).max(axis=1)


# SET LABELS
labels = get_cancellation_date_diff(features)

features.drop(['cancellation_datetime','booking_datetime', 'checkout_date', 'checkin_date'], axis=1, inplace=True)

KeyError: 'cancellation_datetime'

In [91]:
(labels>0.5).sum()

4782

In [68]:
# Seperate training


X_train_regressor, X_test_regressor, y_train_regressor, y_test_regressor = train_test_split(
    features, labels, test_size=0.2, random_state=42)

X_train_classifier, X_test_classifier, y_train_classifier, y_test_classifier = train_test_split(
    features, (labels!=0).astype(int), test_size=0.2, random_state=42)



In [45]:
features.isnull().sum().sort_index()

accommadation_type_name      0
booking_datetime_date_day    0
booking_datetime_month       0
booking_datetime_quarter     0
booking_datetime_weekday     0
booking_datetime_year        0
bus_days                     0
cancellation_policy_code     0
charge_option                0
checkin_date_date_day        0
checkin_date_month           0
checkin_date_quarter         0
checkin_date_weekday         0
checkin_date_year            0
checkout_date_date_day       0
checkout_date_month          0
checkout_date_quarter        0
checkout_date_weekday        0
checkout_date_year           0
customer_nationality         0
guest_is_not_the_customer    0
hotel_area_code              0
hotel_brand_code             0
hotel_chain_code             0
hotel_country_code           0
hotel_star_rating            0
is_first_booking             0
is_user_logged_in            0
no_of_adults                 0
no_of_children               0
no_of_extra_bed              0
no_of_room                   0
origin_c

In [86]:
y_train_regressor.sum()

87692.0

In [84]:
# Training
model_regressor = train_regressor_model(X_train_regressor, y_train_regressor)
print(y_train_regressor)
model_classifier = train_classifier_model(X_train_classifier, y_train_classifier)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


54348    0.0
19312    0.0
12945    0.0
48989    0.0
23534    0.0
        ... 
54343    0.0
38158    0.0
860      0.0
15795    0.0
56422    0.0
Length: 46927, dtype: float64


In [83]:
# Model accuracy
y_class_pred= model_classifier.predict(X_test_classifier)
# print(f'Accuracy of classifier model: {accuracy_score(y_test_classifier, y_pred)}')
y_reg_predict = model_regressor.predict(X_test_regressor)>0.5
# accuracy of regressor model
print(f'Accuracy of regressor model: {accuracy_score(y_test_regressor, y_reg_predict)}')
# sklearn.metrics.mean_squared_error(y_test_regressor, model_regressor.predict(X_test_regressor))

print(model_regressor.predict(X_test_regressor))


Accuracy of regressor model: 0.8880838731674053
[-1013.31652248  -594.0407381   -594.0407381  ...  -635.70032601
 -9636.64985436 -3625.37044564]


In [120]:
# Load data
train_set = pd.read_csv("../datasets/agoda_cancellation_train.csv")

# train_set.drop(['cancellation_datetime'], axis=1, inplace=True)
features = design_matrix(train_set)
sub_checkout_cancelation = []
labels = features['cancellation_datetime'].isnull().astype(int)
features['cancellation_datetime'] = pd.to_datetime(features['cancellation_datetime'], errors='coerce')
for i in range(len(features)):
    if(pd.notnull(features['cancellation_datetime'][i])):
        sub_checkout_cancelation.append((features['cancellation_datetime'][i]-features['booking_datetime'][i]).dt.days)
    else:
        sub_checkout_cancelation.append(0)
   

booking_datetime
checkout_date
checkin_date
cancellation_datetime


AttributeError: 'Timedelta' object has no attribute 'dt'

In [38]:
# train model
classifier = train_model(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)
# # How many people canceled after one week
# canceled_after_one_week = features[features['sub_checkout_checkin'] == 1]

0.7375554040231844

In [11]:
# load predicted data
predicted = pd.read_csv("predicted_results.csv")
actual = pd.read_csv("actual_results.csv")

actual = actual['label'].str.strip().str[-1].astype(int)

predicted =  predicted.replace({0:1, 1:0})

# calculate predicted accuracy
accuracy_score(actual, predicted)


0.8571428571428571